**Importing required libraries**

In [1]:
# Imporing the required library and reading in the data we want to work with
import pandas as pd
import math
import random

# Functions

**y_end()** function takes a dictionary as an argument which is expected to have one 'y' as a key and returns the same dictionary by placing that key and it's value to the end. This will make job easier for another function that calculates the output (y and hi).

In [2]:
def y_end(d):
    y_val = d.pop("y")
    d["y"] = y_val
    return d

**values()** takes in two dictionaries, first is the input data and second the weights. The input data has the input value for the input layer of the network and is used to calculate the values of hidden layers (the hs) using the weights. These values of the hidden layers are then used with their respective weights to calculate the final output(y). Only after calculating each value for the hidden layer we proceed towards the output layer. This is the reason why we wanted to put 'y' at the end of the dictionary. In each iteration, we calculate the value of each hidden layer/s and append the dictionary and after all the values of the hidden layer is calculated and populated in *vals*, we use that *vals* dictionary to calculate the value of 'y' and finally append that value also to the same dictionary before returning it.

In [3]:
def values(data, weight):
    weight = y_end(weight)
    vals = {}
    for i in weight:
        if i != "y":
            s = weight[i]["b0"]
            for j in data:
                if j != "b0":
                    s = s + data[j]*weight[i][j]
            vals[i] = round(1/(1+math.exp(-s)), 3)
        
        else:
            s = weight[i]["b1"]
            for j in vals:
                if j != "b1":
                    s = s + vals[j]*weight[i][j]
            vals[i] = round(1/(1+math.exp(-s)), 3)
    return vals

**total_error()** function takes the value of y from the dictionary returned by **values()** as first argument and target value as the second argument. This function then calculatess the total error in the network and returns it.

In [4]:
def total_error(y, t):
    return round(0.5*(t-y)**2, 3)

**individual_errors()** is a function that calculates and returns the error for the output(y) and the error for each hidden units (hs). We have three arguments in the function, one is the dictionary that is returned by **values()**, another is the same dictionary of weights and lastly the target value. To calculate the errors for each hidden units we need to first calculate the error for the output. So, from our dictionary which is passed as the first argument in the function, we want to get 'y' first.

Since we appended the output value of 'y' at the end in the **values()** function, we have 'y' at the end in *result*. This is why, while looping, we want to get the last value of the dictionary and calculate the error of that output. After calculation we append that value of the error to *errors* dictionary and we calculate the errors of the hidden units accessing the value of error for the output. The errors of each hidden unit gets appended to *errors* in each iteration.

Finally the dictionary containing error for the output, and for each of the hidden units is returned by the function.

In [5]:
def individual_errors(result, weights, t):
    errors = {}
    for i in range(len(result)-1, -1, -1):
        temp = list(result.keys())[i]
        if temp == 'y':
            errors[temp] = round(result[temp]*(1-result[temp])*(t - result[temp]), 3)
        else:
            errors[temp] = round(result[temp]*(1-result[temp])*(weights["y"][temp] * errors["y"]), 3)
    return errors

**learn()** takes the input values, weights, the returned dictionary of **values()** and the individual errors dictionary returned by **individual_errors()** as four arguments and returns a dictionary that is similar in format as the weights dictionary that gets passed as the argument. This is because the idea is to update the weights that we passed and return new weights. 

First we loop through each output units we have (the actual output and the hidden units). Then within each of these iterations, we calculate the updated weights for all the nodes that reach to that output unit. Since the value for hidden units and inputs are in two different dictionaries, we use try and except statements to access their values. For each 'i' we get one dictionary and we append each of these dictionaries to the *updated_weight* dictionary before returning it.

In [6]:
def learn(inputs, weights, result, ind_errs):
    updated_weights = {}
    for i in ind_errs:
        updated = {}
        for j in weights[i]:
            if j == "b1":
                updated[j] = round(weights[i][j] + (0.5 * ind_errs[i] * 1), 3)
            else:
                try:
                    updated[j] = round(weights[i][j] + (0.5 * ind_errs[i] * result[j]), 3)
                except KeyError:
                    updated[j] = round(weights[i][j] + (0.5 * ind_errs[i] * inputs[j]), 3)
        updated_weights[i] = updated
    return(updated_weights)

All of these functions could have been incorporated in a couple of functions but we decided to split them into all these to make it easier for documentation and to display each calculations.

In [7]:
inputs = {"i1": 0, "i2": 1, "b0" : 1}
weights = {"h1": {"b0": 1, "i1": 1, "i2": 0.5}, "h2": {"b0": 1, "i1": -1, "i2": 2}, "y" : {"b1": 1, "h1": 1.5, "h2": -1}}

The two dictionaries 'inputs' and 'weights' hold the input values and weight values from the tutorial. Using the **values()** function we *feed the inputs forward* and check the total error of the model using the **total_error()** function. 

We can further reduce the error if we *backpropagate the errors* to go back and adjust the weights. We calculate the individual error of the output and hidden units with the **individual_errors()** function and use these errors to update the weights with the **learn()** function. 

After executing the **learn()** function, we get the updated weights which we can then use to recalulate the value of y and to check the network error which should now be less than what we had before.

We can repeat this process multiple times in an effort to reduce the error and get new weights value that can better predict the output.

In [8]:
result = values(inputs,weights)
result

{'h1': 0.818, 'h2': 0.953, 'y': 0.781}

In [9]:
totals = total_error(result['y'], 1)
totals

0.024

In [10]:
ind_errs = individual_errors(result, weights, 1)
ind_errs

{'y': 0.037, 'h2': -0.002, 'h1': 0.008}

In [11]:
weights = learn(inputs, weights, result, ind_errs)
weights

{'y': {'b1': 1.018, 'h1': 1.515, 'h2': -0.982},
 'h2': {'b0': 0.999, 'i1': -1.0, 'i2': 1.999},
 'h1': {'b0': 1.004, 'i1': 1.0, 'i2': 0.504}}

The above code chunk gave us the updated weights which reduces the total network error. Let's check it by using the updated weights to the same input and target data.

In [12]:
result = values(inputs,weights)
totals = total_error(result['y'], 1)
ind_errs = individual_errors(result, weights, 1)
weights = learn(inputs, weights, result, ind_errs)
print(result)
print(totals)

{'h2': 0.952, 'h1': 0.819, 'y': 0.79}
0.022


As we can see, the value of y has gotten closer to 1 and the error has also reduced from 0.024 to 0.022.